In [1]:
# === Imports ===
import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf
from pandas_datareader import data as web
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
# === Parameters ===
today = dt.datetime.today()
start = today - dt.timedelta(days=365)
T = 0.5  # 6 months to maturity
X = 1.25  # Strike price
d = 0.01  # Storage cost

In [3]:
# === 1. Retrieve Data ===
# 1A. Spot Price of Coffee (use last close)
coffee_data = yf.download('KC=F', start=start, end=today)
S_0 = coffee_data['Close'].dropna().iloc[-1]

# 1B. Risk-Free Rate (6-month Treasury)
r_data = web.DataReader('DTB6', 'fred', start, today)
r = r_data.dropna().iloc[-1].values[0] / 100  # Convert % to decimal

# 1C. Historical Volatility (Annualized)
log_returns = np.log(coffee_data['Close'] / coffee_data['Close'].shift(1)).dropna()
sigma = log_returns.std() * np.sqrt(252)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [4]:
# === 2. Cost of Carry Model ===
F_t = S_0 * np.exp((r + d) * T)

In [5]:
# === 3. Black-Scholes Option Pricing ===
d1 = (np.log(S_0 / X) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)
C = S_0 * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)

In [6]:
# === 4. Monte Carlo Simulation ===
num_simulations = 10000
num_steps = 252
dt = T / num_steps
np.random.seed(42)
price_paths = np.zeros((num_steps, num_simulations))
price_paths[0] = np.full(num_simulations, S_0)

for t in range(1, num_steps):
    z = np.random.standard_normal(num_simulations)
    price_paths[t] = price_paths[t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

final_prices = price_paths[-1]
average_simulated_price = np.mean(final_prices)

ValueError: Length of values (10000) does not match length of index (1)

In [ ]:
# === Results ===
print(f"Spot Price: ${S_0:.3f}")
print(f"Risk-Free Rate: {r:.2%}")
print(f"Volatility: {sigma:.2%}")
print(f"Futures Price (Cost of Carry): ${F_t:.3f}")
print(f"Call Option Price (Black-Scholes): ${C:.3f}")
print(f"Average Simulated Price at Maturity (Monte Carlo): ${average_simulated_price:.3f}")

# === Optional: Plot a sample of simulations ===
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.plot(price_paths[:, i], lw=0.5)
plt.title("Sample Monte Carlo Price Paths for Coffee Futures")
plt.xlabel("Time Step")
plt.ylabel("Price ($)")
plt.grid(True)
plt.show()
